# AntIzetomic analysis of perivascular cells

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy

from anndata import AnnData
import scanpy as sc
import scanpy.external as sce
import scvelo as scv

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

from scripts.batch_process import batch_process, preprocess_adata_sub
from cellassign import assign_cats

In [ ]:
seed = 10
sc.set_figure_params(dpi=200, dpi_save=300)

In [ ]:
from scipy.stats import spearmanr

In [ ]:
# Selection of palettes for cluster coloring, and scatter values

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

dict_colors_pops = {'#E91E63': ['PC1D0', 'PC1D2', 'PC1D4'],  '#FF5722': ['PC2D0', 'PC2D2', 'PC2D4'], 
                    '#FFC107': ['PC3D0', 'PC3D2', 'PC3AD4'], '#FFEB3B': ['PC3BD4'], 
                    '#8BC34A': ['PC4D0', 'PC4D2', 'PC4D4'],  '#2196F3': ['PC5D0', 'PC5D2', 'PC5D4'],
                    '#673AB7': ['PC6D0', 'PC6D2', 'PC6D4'],}

dict_pops_colors = {x: k for k,v in dict_colors_pops.items() for x in v }     

# AnnData loading

In [ ]:
dir_adata_save = os.getcwd() + '/data/preprocessed/'

In [ ]:
adata_D0 = sc.read_h5ad(dir_adata_save + '/A+M_0.h5ad')

In [ ]:
adata_D0

In [ ]:
sc.pl.umap(adata_D0, color=['batch', 'cell_types'], ncols=2)

In [ ]:
adata_D2 = sc.read_h5ad(dir_adata_save + '/A+M_2.h5ad')

In [ ]:
sc.pl.umap(adata_D2, color=['batch', 'cell_types'], ncols=2)

In [ ]:
adata_D4 = sc.read_h5ad(dir_adata_save + '/A+M_4.h5ad')

In [ ]:
sc.pl.umap(adata_D4, color=['batch', 'cell_types'], ncols=2)

In [ ]:
for adata in [adata_D0, adata_D2, adata_D4,]:
    adata.obs_names = [i.replace('-1-', '-') for i in adata.obs_names]

# Discovering cell subtypes

In [ ]:
cell_type = 'Perivascular'

D0_sub = adata_D0[adata_D0.obs['cell_types'] == cell_type]
D2_sub = adata_D2[adata_D2.obs['cell_types'] == cell_type]
D4_sub = adata_D4[adata_D4.obs['cell_types'] == cell_type]

for adata in [D0_sub, D2_sub, D4_sub]:
    adata.obs['donor'] = [i[0] for i in adata.obs['batch']]

## Preprocessing of day 0

In [ ]:
preprocess_adata_sub(D0_sub, resolution=0.7, n_HVG=700, min_dist=0.5, seed=seed)
sc.pl.umap(D0_sub, color=['batch', 'leiden'])

In [ ]:
dict_peri_D0 = {'PC1D0': ['PCP4', 'CNN1', 'DES'],   # P5D0
                'PC2D0': ['RERGL', 'BCAM', 'ADIRF'],   # P3D0
                'PC3D0': ['CYCS', 'CREM', 'HSPD1'],   # P0D0
                'PC4D0': ['RGS5', 'POSTN', 'KCJN8'],   # P2D0
                'PC5D0': ['SSTR2', 'FGF7', 'EDNRB'],   # P4D0 
                'PC6D0': ['CCL19', 'CXCL12', 'CCL2'],}   # P1D0

assign_cats(D0_sub, dict_cats=dict_peri_D0, key_added='cell_subcats', min_score=0.5, quantile_gene_sel=0.8)
D0_sub.uns['cell_subcats_colors'] = [dict_pops_colors[i] for i in sorted(set(D0_sub.obs['cell_subcats']))]

sc.pl.umap(D0_sub, color=['leiden', 'cell_subcats'])

In [ ]:
sc.tl.rank_genes_groups(D0_sub, groupby='leiden', groups=['4'], reference='rest', method='wilcoxon')
sc.pl.rank_genes_groups_tracksplot(D0_sub, dendrogram=False, n_genes=50, use_raw=False)

In [ ]:
D0_sub.write_h5ad(f'{dir_adata_save}/Peri_D0.h5')

## Preprocessing of day 2

In [ ]:
preprocess_adata_sub(D2_sub, resolution=2, n_HVG=1000, min_dist=0.2, seed=seed)
sc.pl.umap(D2_sub, color=['batch', 'leiden'])

In [ ]:
dict_peri_D2 = {'PC1D2': ['PCP4', 'CNN1', 'DES'],  # P4D2
                'PC2D2': ['RERGL', 'BCAM', 'ADIRF'],  # P1D2
                'PC3D2': ['CYCS', 'CREM', 'HSPD1'],  # P3D2
                'PC4D2': ['RGS5', 'ABCC9', 'NCKAP5'],  # P0D2
                'PC5D2': ['SOD2', 'GJA4', 'SSTR2'],  # Didn't exist
                'PC6D2': ['CCL19', 'CXCL12', 'CCL2'],}  # P2D2

assign_cats(D2_sub, dict_cats=dict_peri_D2, key_added='cell_subcats', min_score=0.5, quantile_gene_sel=0.99)
D2_sub.uns['cell_subcats_colors'] = [dict_pops_colors[i] for i in sorted(set(D2_sub.obs['cell_subcats']))]

sc.pl.umap(D2_sub, color=['leiden', 'cell_subcats'])

In [ ]:
sc.pl.umap(D2_sub, color=['leiden', 'RGS5', 'RERGL', 'CCL19', 'CYCS', 'DES', 'SSTR2', 'FGF7'], cmap=magma, use_raw=False)

In [ ]:
sc.tl.rank_genes_groups(D2_sub, groupby='leiden', groups=['5'], reference='rest', method='wilcoxon')
sc.pl.rank_genes_groups_tracksplot(D2_sub, dendrogram=False, n_genes=50, use_raw=False)

In [ ]:
D2_sub.write_h5ad(f'{dir_adata_save}/Peri_D2.h5')

## Preprocessing of day 4

In [ ]:
preprocess_adata_sub(D4_sub, resolution=1, n_HVG=1500, min_dist=0.2, seed=seed)
sc.pl.umap(D4_sub, color=['batch', 'leiden'])

In [ ]:
dict_peri_D4 = {'PC1D4': ['TPM1', 'DES', 'PCP4'],  # P4D4
                'PC2D4': ['RERGL', 'MUSTN1', 'SDF4'],  # P1D4
                'PC3AD4': ['CYCS', 'CREM', 'MT1A'],  # P0D4
                'PC3BD4': ['IL6', 'INHBA', 'PDK4'],  # P6D4
                'PC4D4': ['RGS5', 'ABCC9', 'NDUFA4L2'],  # P2D4
                'PC5D4': ['FGF7', 'SSTR2', 'SGIP1'],  # P5D4
                'PC6D4': ['CCL19', 'CCL2', 'CCL21'],}  # P3D4

assign_cats(D4_sub, dict_cats=dict_peri_D4, key_added='cell_subcats', min_score=0.5, quantile_gene_sel=0.99)
D4_sub.uns['cell_subcats_colors'] = [dict_pops_colors[i] for i in sorted(set(D4_sub.obs['cell_subcats']))]

sc.pl.umap(D4_sub, color=['leiden', 'cell_subcats'])

In [ ]:
sc.tl.rank_genes_groups(D4_sub, groupby='cell_subcats', groups=['PC3BD4'], reference='rest', method='wilcoxon')
sc.pl.rank_genes_groups_tracksplot(D4_sub, dendrogram=False, n_genes=50, use_raw=False)

In [ ]:
D4_sub.write_h5ad(f'{dir_adata_save}/Peri_D4.h5')